In [3]:
import pandas as pd
bicis = pd.read_excel("ny_citibikes_raw.xlsx")

# 1. Exploración del conjunto de datos
  - ¿Qué información contiene cada columna?
  

In [4]:
bicis.head()

,Start Time,Stop Time,Start Station ID,Start Station Name,End Station ID,End Station Name,Bike ID,User Type,Birth Year,Age,Age Groups,Trip Duration,Trip_Duration_in_min,Month,Season,Temperature,Weekday
0,2017-01-01 00:38:00,2017-01-01 01:03:00,3194,McGinley Square,3271,Danforth Light Rail,24668,Subscriber,1961,60,55-64,1513,25,1,Winter,10,Sunday
1,2017-01-01 01:47:00,2017-01-01 01:58:00,3183,Exchange Place,3203,Hamilton Park,26167,Subscriber,1993,28,25-34,639,11,1,Winter,10,Sunday
2,2017-01-01 01:47:00,2017-01-01 01:58:00,3183,Exchange Place,3203,Hamilton Park,26167,Subscriber,1993,28,25-34,639,11,1,Winter,10,Sunday
3,2017-01-01 01:56:00,2017-01-01 02:00:00,3186,Grove St PATH,3270,Jersey & 6th St,24604,Subscriber,1970,51,45-54,258,4,1,Winter,10,Sunday
4,2017-01-01 02:12:00,2017-01-01 02:23:00,3270,Jersey & 6th St,3206,Hilltop,24641,Subscriber,1978,43,35-44,663,11,1,Winter,10,Sunday


- ¿Hay valores faltantes o duplicados?

Solo hay uno.

In [5]:
bicis.isna().sum()

Start Time              0
Stop Time               0
Start Station ID        0
Start Station Name      0
End Station ID          0
End Station Name        1
Bike ID                 0
User Type               0
Birth Year              0
Age                     0
Age Groups              0
Trip Duration           0
Trip_Duration_in_min    0
Month                   0
Season                  0
Temperature             0
Weekday                 0
dtype: int64

- ¿Cuál es el lapso de tiempo total de los viajes?

In [6]:
bicis["Trip_Duration_in_min"].head()

0    25
1    11
2    11
3     4
4    11
Name: Trip_Duration_in_min, dtype: int64

In [7]:
bicis["Duracion Tiempo"]= pd.to_datetime(bicis["Stop Time"]) - pd.to_datetime(bicis["Start Time"])

bicis["Duracion Tiempo"].head()

0   0 days 00:25:00
1   0 days 00:11:00
2   0 days 00:11:00
3   0 days 00:04:00
4   0 days 00:11:00
Name: Duracion Tiempo, dtype: timedelta64[ns]

# 2. Estadísticas básicas
  - ¿Cuál es la duración media del viaje (en minutos)?
  



In [8]:
print(f'Media: {bicis["Duracion Tiempo"].mean().total_seconds() / 60} minutos')

Media: 9.409362733333333 minutos


- ¿Cuál es la duración mínima y máxima?
 

In [18]:
print(f'Mínimo: {bicis["Duracion Tiempo"].min()}')

print(f'Máximo: {bicis["Duracion Tiempo"].max()}')

Mínimo: 0 days 00:01:00
Máximo: 4 days 12:35:00


 - ¿Cuáles son las estaciones de inicio y fin más comunes?

In [10]:
print(f'Início: {bicis["Start Station ID"].value_counts().max()}')

print(f'Fin: {bicis["End Station ID"].value_counts().max()}')

Início: 2544
Fin: 3313


# 3. Usuarios y datos demográficos
  - ¿Cuántas bicicletas únicas se utilizaron?
  

In [11]:
print(f'Numero de bícis únicas: {len(bicis["Bike ID"].unique())}')

Numero de bícis únicas: 500


- ¿Cuál es la proporción de cada tipo de usuario (suscriptor vs. cliente)?

In [12]:
print(f'Cantidad Subscriptores: {bicis["User Type"].value_counts(normalize=True)["Subscriber"]}')

print(f'Cantidad One-time user: {bicis["User Type"].value_counts(normalize=True)["One-time user"]}')

Cantidad Subscriptores: 0.9813725490196078
Cantidad One-time user: 0.018627450980392157



  - ​​¿Cuál es la distribución por edades de los usuarios? ¿Qué grupo de edad utiliza más el servicio?

In [13]:
print(f'Gupo edades: {bicis["Age Groups"].unique()}')

print(f'{bicis.groupby("Age Groups")['Age'].count()}')

Gupo edades: ['55-64' '25-34' '45-54' '35-44' '65-74' '18-24' '75+']
Age Groups
18-24      60
25-34    4936
35-44    9247
45-54    3581
55-64    1749
65-74     760
75+        67
Name: Age, dtype: int64


# 4. Análisis temporal
  - ¿Cómo varía el número de viajes según el día de la semana?




In [14]:
print(f'{bicis["Weekday"].value_counts(normalize=True)}')



Weekday
Wednesday    0.196176
Thursday     0.178186
Monday       0.149559
Friday       0.145000
Tuesday      0.143971
Sunday       0.093873
Saturday     0.093235
Name: proportion, dtype: float64


  - ¿Qué mes o temporada tiene la mayor cantidad de viajes?

In [15]:
print(f'{bicis["Season"].value_counts()} \n')

print(f'{bicis["Month"].value_counts()}')

Season
Winter    10742
Spring     9658
Name: count, dtype: int64 

Month
3    9658
2    5604
1    5138
Name: count, dtype: int64


 - ¿A qué hora del día comienzan la mayoría de los viajes?

In [20]:
bicis["Hora"] = bicis["Start Time"].dt.hour

hora_mas_frecuente = bicis["Hora"].value_counts().idxmax()
print(f"La hora con más viajes es: {hora_mas_frecuente}:00")

La hora con más viajes es: 8:00


# 5. Análisis geográfico
  - ¿Qué pares de estaciones (inicio → fin) aparecen con mayor frecuencia?

In [17]:
bicis[["Start Station ID","End Station ID"]].value_counts().head()

Start Station ID  End Station ID
3203              3186              500
3267              3183              445
3183              3267              367
3279              3186              349
3186              3203              340
Name: count, dtype: int64


  - ¿Hay alguna estación que aparezca solo como estación de inicio o solo como estación de fin?

In [22]:
bicis[["Start Station ID","End Station ID"]].value_counts().isna().sum()

np.int64(0)

# 6. Temperatura y duración
  - ¿Existe alguna relación visible entre la temperatura y la duración del viaje?



In [29]:
bicis[["Trip Duration","Temperature"]].value_counts().head()


Trip Duration  Temperature
240            16             28
239            16             22
260            16             20
235            16             20
212            16             19
Name: count, dtype: int64

  - ¿Cómo varía la duración media del viaje según la temporada?

In [37]:
bicis.groupby("Season")["Trip Duration"].mean()

Season
Spring    568.525885
Winter    559.632191
Name: Trip Duration, dtype: float64

# 7. Resumen e interpretación
 - Escriba un breve resumen (5-10 líneas) de sus hallazgos.
 - Mencione patrones, anomalías o tendencias interesantes que haya observado.